# Predicting House Prices (Keras - Artificial Neural Network)

### Setups

In [2]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt

# scaling and train test split
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

# creating a model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam

# evaluation on test data
from sklearn.metrics import mean_squared_error,mean_absolute_error,explained_variance_score
from sklearn.metrics import classification_report,confusion_matrix

### Constans

In [3]:
DATASETS_DIR = '/Users/SISTEMAS/MLOPs_Project/DataSet/kc_house_data.csv'
COLUMNS_TO_DROP = ['id', 'zipcode', 'date']

### Custum Transformers

In [5]:
class CustomTransformer(BaseEstimator, TransformerMixin):
    #the constructor
    '''setting the add_bedrooms_per_room to True helps us check if the hyperparameter is useful'''
    def __init__(self, add_bedrooms_per_room = True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
    #estimator method
    def fit(self, X, y = None):
        return self
    #transfprmation
    def transform(self, X, y = None):
        #agregar 2 columnas
        X_copy = X.copy()
        X_copy['date'] = pd.to_datetime(X_copy['date'])
        X_copy['month'] = X_copy['date'].apply(lambda date: date.month)
        X_copy['year'] = X_copy['date'].apply(lambda date: date.year)
        #X_copy = X_copy.drop('date', axis=1)
        return X_copy

    
#Agregar_Caracteristicas = CustomTransformer()
#DataSet = Agregar_Caracteristicas.transform(df)  


In [6]:
class DropColumnsTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.COLUMNS_TO_DROP = COLUMNS_TO_DROP
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_copy = X.drop(self.COLUMNS_TO_DROP, axis=1)
        return X_copy

# Instanciar el custom transformer
#drop_columns_transformer = DropColumnsTransformer(COLUMNS_TO_DROP)
    
    # Transformar los datos
#transformed_data = drop_columns_transformer.transform(df)


In [7]:
class CustomMinMaxScaler(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.scaler = MinMaxScaler()
        
    def fit(self, X, y=None):
        # Ajusta el escalador en los datos de entrenamiento
        self.scaler.fit(X)
        return self
    
    def transform(self, X):
        # Transforma los datos usando el escalador ajustado
        X_scaled = self.scaler.transform(X)
        return X_scaled


### PIPELINE

In [8]:
df = pd.read_csv(DATASETS_DIR)

In [9]:
House_Price_Pipeline = Pipeline([
        ('Agregar_Variables',CustomTransformer()),
        ('DropColumns',DropColumnsTransformer()),
        ])

In [10]:
df = House_Price_Pipeline.fit_transform(df)  

In [11]:

X_train, X_test, y_train, y_test = train_test_split(df.drop('price',axis=1),
                                                    df['price'],test_size=0.3,random_state=101
                                                    )


In [12]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(15129, 19)
(6484, 19)
(15129,)
(6484,)


In [13]:
scaler = MinMaxScaler()

# fit and transfrom
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#preguntar-MinMaxScaler_Pipeline = Pipeline([
 #       ('Scalar',CustomMinMaxScaler()),
  #      ])

#X_train = MinMaxScaler_Pipeline.fit_transform(X_train)        

<a id="ch7"></a>
## Creating a model
***
We estimate the number of neurons (units) from our features. Ex: X_train.shape (15117, 19). The optimizer is asking how you want to perform this gradient descent. In this case we are using the Adam optimizer and the mean square error loss function.

In [15]:
model = Sequential()

# input layer
model.add(Dense(19,activation='relu'))

# hidden layers
model.add(Dense(19,activation='relu'))
model.add(Dense(19,activation='relu'))
model.add(Dense(19,activation='relu'))

# output layer
model.add(Dense(1))

model.compile(optimizer='adam',loss='mse')


## Training the model
Now that the model is ready, we can fit the model into the data.

Since the dataset is large, we are going to use batch_size. It is typical to use batches of the power of 2 (32, 64, 128, 256...). In this case we are using 128. The smaller the batch size, the longer is going to take.

In [323]:
model.fit(x=X_train,y=y_train.values,
          validation_data=(X_test,y_test.values),
          batch_size=128,epochs=400)

Epoch 1/400
119/119 [==============================] - 1s 4ms/step - loss: 423609434112.0000 - val_loss: 432928194560.0000
Epoch 2/400
119/119 [==============================] - 0s 3ms/step - loss: 421928337408.0000 - val_loss: 427347673088.0000
Epoch 3/400
119/119 [==============================] - 0s 2ms/step - loss: 401891033088.0000 - val_loss: 382281121792.0000
Epoch 4/400
119/119 [==============================] - 0s 3ms/step - loss: 312255741952.0000 - val_loss: 241216552960.0000
Epoch 5/400
119/119 [==============================] - 0s 3ms/step - loss: 160572080128.0000 - val_loss: 116006772736.0000
Epoch 6/400
119/119 [==============================] - 0s 3ms/step - loss: 101370503168.0000 - val_loss: 104035074048.0000
Epoch 7/400
119/119 [==============================] - 0s 3ms/step - loss: 97954070528.0000 - val_loss: 102292463616.0000
Epoch 8/400
119/119 [==============================] - 0s 3ms/step - loss: 96373039104.0000 - val_loss: 100629078016.0000
Epoch 9/400
119/11

<a id="ch9"></a>
## Evaluation on test data
---
### Regression Evaluation Metrics

**Mean Absolute Error** (MAE) is the mean of the absolute value of the errors:

$$\frac 1n\sum_{i=1}^n|y_i-\hat{y}_i|$$

**Mean Squared Error** (MSE) is the mean of the squared errors:

$$\frac 1n\sum_{i=1}^n(y_i-\hat{y}_i)^2$$

**Root Mean Squared Error** (RMSE) is the square root of the mean of the squared errors:

$$\sqrt{\frac 1n\sum_{i=1}^n(y_i-\hat{y}_i)^2}$$

Comparing these metrics:

- **MAE** is the easiest to understand, because it's the average error.
- **MSE** is more popular than MAE, because MSE "punishes" larger errors, which tends to be useful in the real world.
- **RMSE** is even more popular than MSE, because RMSE is interpretable in the "y" units.

### Predicting on brand new data
In this part we are giving the model the test set to get a list of predictions. Then we compare the correct values with the list of predictions. We use different metrics to compare the predictions, in this case we use MAE, MSE, RMSE and Variance Regression Score.

Let us start by analyzing the MAE, which is \\$103,500. This means that our model is off on average about \\$100,000.

***Is that MAE good or bad?***

For that we must take into account our original data set and see what kind of values we have. For instance, the mean is 540,000, therefore the MEA is about 19% of the mean price. This is not a particularly good result.

To better understand this error, we can use the variance regression score, where the best possible score is 1.0 and lower values are worse. This tells you how much variance is being explain by your model. In our case we have 0.80 which is a normal result.

In [325]:
# predictions on the test set
predictions = model.predict(X_test)

print('MAE: ',mean_absolute_error(y_test,predictions))
print('MSE: ',mean_squared_error(y_test,predictions))
print('RMSE: ',np.sqrt(mean_squared_error(y_test,predictions)))
print('Variance Regression Score: ',explained_variance_score(y_test,predictions))

print('\n\nDescriptive Statistics:\n',df['price'].describe())

203/203 [==============================] - 0s 1ms/step
MAE:  104101.97558172038
MSE:  28573731467.21278
RMSE:  169037.66286603935
Variance Regression Score:  0.7964006601429703


Descriptive Statistics:
 count    2.161300e+04
mean     5.400881e+05
std      3.671272e+05
min      7.500000e+04
25%      3.219500e+05
50%      4.500000e+05
75%      6.450000e+05
max      7.700000e+06
Name: price, dtype: float64


### Persisting the trained model

In [334]:
import joblib

TRAINED_MODEL_DIR = '/Users/SISTEMAS/MLOPS-PROJECT/house_price_prediction/house_price_prediction/models/'
# Guardar la red neuronal en un archivo .pkl
FILE_NAME = 'neural_network_model.pkl'
joblib.dump(model, TRAINED_MODEL_DIR+FILE_NAME)
print(f"Modelo guardado en {TRAINED_MODEL_DIR+FILE_NAME}")

Modelo guardado en /Users/SISTEMAS/MLOPS-PROJECT/house_price_prediction/house_price_prediction/models/neural_network_model.pkl


## Predicting on a brand new house
---
We are going to use the model to predict the price on a brand-new house. We are going to choose the first house of the data set and drop the price. single_house is going to have all the features that we need to predict the price. After that we need to reshape the variable and scale the features.

The original price is \\$221,900 and the model prediction is \\$280,000.

In [336]:
# fueatures of new house
single_house = df.drop('price',axis=1).iloc[0]
print(f'Features of new house:\n{single_house}')

# reshape the numpy array and scale the features
single_house = scaler.transform(single_house.values.reshape(-1, 19))

# run the model and get the price prediction
print('\nPrediction Price:',model.predict(single_house)[0,0])

# original price
print('\nOriginal Price:',df.iloc[0]['price'])

Features of new house:
bedrooms            3.0000
bathrooms           1.0000
sqft_living      1180.0000
sqft_lot         5650.0000
floors              1.0000
waterfront          0.0000
view                0.0000
condition           3.0000
grade               7.0000
sqft_above       1180.0000
sqft_basement       0.0000
yr_built         1955.0000
yr_renovated        0.0000
lat                47.5112
long             -122.2570
sqft_living15    1340.0000
sqft_lot15       5650.0000
month              10.0000
year             2014.0000
Name: 0, dtype: float64
1/1 [==============================] - 0s 24ms/step

Prediction Price: 270773.97

Original Price: 221900.0


c:\Users\SISTEMAS\MLOPS-PROJECT\.venv\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
